In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
# ต้องใช้ google colab เพราะ IP เครื่อง ไม่มี Permission

ข้อมูลการใช้รถไฟในรัฐนิวยอร์ก โดยพิจารณาจากประตูทางเข้า (trunstile)

https://data.ny.gov/Transportation/MTA-Subway-Hourly-Ridership-Beginning-February-202/wujg-7c2s

https://data.ny.gov/w/9fh6-zjuj/caer-yrtv?cur=XOXcPm0W3AQ&from=cyZwA7fO28p

In [2]:
# Set the URL you want to webscrape from
url = 'http://web.mta.info/developers/turnstile.html'
# Connect to the URL
response = requests.get(url)

ConnectionError: HTTPConnectionPool(host='web.mta.info', port=80): Max retries exceeded with url: /developers/turnstile.html (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1062c1b20>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

ดึงข้อมูลจาก url เดิม ผ่านคำสั่ง requests.get อย่างไรก็ตามหากต้องการเข้า  url นี้โดยตรงจะไม่สามารถเข้าได้แล้วในปัจจุบัน

In [ ]:
print(response.text)

<HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;web&#46;mta&#46;info&#47;developers&#47;turnstile&#46;html" on this server.<P>
Reference&#32;&#35;18&#46;15f39b7a&#46;1725354071&#46;31cdfc73
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;15f39b7a&#46;1725354071&#46;31cdfc73</P>
</BODY>
</HTML>



In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup.findAll('a')

[]

In [ ]:
# To download the data set, let's do a tag
one_a_tag = soup.findAll('a')[37]
link = one_a_tag['href']
print(link)

IndexError: list index out of range

In [ ]:
download_url = 'http://web.mta.info/developers'+ link
print(download_url)

In [ ]:
#! wget http://web.mta.info/developers/data/nyct/turnstile/turnstile_200926.txt
! wget http://web.mta.info/developers/data/nyct/turnstile/turnstile_171125.txt
! wget http://web.mta.info/developers/data/nyct/turnstile/turnstile_231216.txt
#download file and save to colab

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
! ls -la

In [ ]:
! mv 'turnstile_171125.txt' '/drive/My Drive/turnstile_171125.txt'

In [ ]:
! cp 'turnstile_171125.txt' '/drive/My Drive/turnstile_231216.txt'

In [ ]:
! ls -la

Turnstile data consists of
*   C/A = Control Area (e.g., A002)
*   unit = Remote Unit for a station (e.g., R051)
*   SCP = Subunit Channel Position represents an specific address for a device (e.g., 02-00-00)
*   station_code = C/A + unit, locating a station
*   turnstile = C/A + unit + SCP, locating a turnstile
*   Station = Represents the station name the device is located at
*   date = Represents the date (MM-DD-YY)
*   time = Represents the time (hh:mm:ss) for a scheduled audit event
*   datetime = date + time (MM-DD-YY hh:mm:ss)
*   desc = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
*   entries = The comulative entry register value for a device
*   exits = The cumulative exit register value for a device

การเข้าดูโครงสร้างข้อมูลเบื้องต้น

In [ ]:
import pandas as pd
import numpy as np

mta = pd.read_csv('turnstile_231216.txt', sep = ',')
mta.head()

In [ ]:
mta.info()

In [ ]:
mta.columns

สร้างคอลัมน์เกี่ยวกับวันเวลาเพื่อเตรียมทำ Time series

In [ ]:
mta['datetime'] = pd.to_datetime(mta.DATE + ' ' + mta.TIME, format='%m/%d/%Y  %H:%M:%S')

In [ ]:
mta['turnstile'] = mta['C/A'] + '-' + mta['UNIT'] + '-' + mta['SCP']

mta = mta[(mta.datetime >= '2023-12-09 00:00:00') &
          (mta.datetime <'2023-12-10 00:00:00')]